# Invoke AgentCore Runtime (Simple)
This notebook invokes the AgentCore runtime using the same approach as Lab 4.


In [ ]:
# Config
RUNTIME_ARN = "arn:aws:bedrock-agentcore:us-east-2:729665431827:runtime/awslegalpoc_customer_support-NmZatwFYo5"
REGION = "us-east-2"
COGNITO_SECRET_NAME = "awslegalpoc/cognito-config"
USERNAME = "admin"  # set your Cognito username
PASSWORD = "ChangeMe123!"  # set your Cognito password


In [ ]:
import json
import uuid
import boto3
from bedrock_agentcore_starter_toolkit import Runtime
import base64, hmac, hashlib

def _secret_hash(username: str, client_id: str, client_secret: str) -> str:
    message = bytes(username + client_id, "utf-8")
    key = bytes(client_secret, "utf-8")
    return base64.b64encode(hmac.new(key, message, digestmod=hashlib.sha256).digest()).decode()

sm = boto3.client("secretsmanager", region_name=REGION)
secret = sm.get_secret_value(SecretId=COGNITO_SECRET_NAME)
cfg = json.loads(secret["SecretString"])

cognito = boto3.client("cognito-idp", region_name=REGION)
auth = cognito.initiate_auth(
    ClientId=cfg["client_id"],
    AuthFlow="USER_PASSWORD_AUTH",
    AuthParameters={
        "USERNAME": USERNAME,
        "PASSWORD": PASSWORD,
        "SECRET_HASH": _secret_hash(USERNAME, cfg["client_id"], cfg["client_secret"]),
    },
)
access_token = auth["AuthenticationResult"]["AccessToken"]
print("Authenticated. Token length:", len(access_token))


In [ ]:
# Invoke runtime
runtime = Runtime()
session_id = str(uuid.uuid4())
payload = {"prompt": "List all of your tools", "actor_id": "customer_001"}

response = runtime.invoke(
    payload,
    session_id=session_id,
    bearer_token=access_token,
)
print("Session ID:", session_id)
print(response.get("response", ""))
